# Jogo dos 8
### Método de busca não-informada: Busca em Profundidade

In [1]:
import numpy as np
import time
import sys

### Metódo encontraEstadoAtual
O método encontra as coordenadas do valor 0 na matriz passada por parâmetro.

##### Parâmetros: 
    matriz: matriz numpy a ser procurada.
 
##### Retorno: 
    bool: array com coordenadas do valor 0. 

In [2]:
def encontraEstadoAtual(matriz):
    result = np.where(matriz == 0)
    estado_atual = (result[0].item(0), result[1].item(0))
    
    return estado_atual

### Metódo encontraSucessores
Verifica as movimentações válidas do valor 0 na matriz - vertical, horizontal - e retorna os sucessores da matriz de entrada - uma matriz para cada possível movimentação.

##### Parâmetros: 
    matriz: matriz numpy a ser encontrado seus sucessores.
    
    estado_atual: coordenadas do valor 0.

##### Retorno: 
    sucessores: lista com uma matriz para cada possível movimentação do 0.

In [3]:
def encontraSucessores(matriz, estado_atual):
    sucessores = []
    direcoes = [[1, 1], [1, -1], [0, 1], [0, -1]]
    
    for direcao in direcoes:
        matriz_sucessora = np.copy(matriz)
        
        if direcao[0] == 1: # movimentações na vertical
            estado_trocado = (estado_atual[0], estado_atual[1] + direcao[1])
        else: # movimentações na horizontal
            estado_trocado = (estado_atual[0] + direcao[1], estado_atual[1])

        if estado_trocado[direcao[0]] >= 0 and estado_trocado[direcao[0]] < 3: 
            matriz_sucessora[estado_atual] = matriz[estado_trocado]
            matriz_sucessora[estado_trocado] = 0
            sucessores.append(matriz_sucessora)
            
    return sucessores

### Metódo verificaSeFoiVisitado
Verifica se aquele estado da matriz - com os valores nas respectivas posições - já foi visitado.

##### Parâmetros: 
    matriz: matriz numpy a ser verificada.
    
    estados_visitados: lista com matrizes visitadas.

##### Retorno: 
    bool: True se já foi visitado, False se não foi.

In [4]:
def verificaSeFoiVisitado(matriz, estados_visitados):
    for estado in estados_visitados:
        if (matriz == estado).all():
            return True
        
    return False

### Metódo acharProximoSucessorNaoVisitado
Verifica o próximo sucessor que não foi visitado - "voltando" na árvore. Essa função é usada quando todos sucessores do nó atual já foram visitados.

##### Parâmetros: 
    estados_visitados: lista com todas matrizes já visitadas.
    
    estados_com_sucessores_nv: lista com todas matrizes com sucessores que ainda não foram verificados, e possívelmente podem ter sucessores não visitados.
    
    sucessores: sucessores do nó atual.
    
    index_atual: index na lista de sucessores que o nó atual está.

##### Retorno: 
    sucessor: estado ainda não visitado. 

In [5]:
def acharProximoSucessorNaoVisitado(estados_visitados, estados_com_sucessores_nv, sucessores, index_atual):
    while len(sucessores) > index_atual + 1:
        ja_visitado = False
        for estado in estados_visitados:
            if (sucessores[index_atual + 1] == estado).all(): 
                ja_visitado = True
        
        if ja_visitado:
            index_atual += 1
        else:
            return sucessores[index_atual + 1]
    
    estado = estados_com_sucessores_nv[-1]
    estado_com_sucessores_nao_visitados = np.copy(estados_com_sucessores_nv[:-1])
    
    sucessores_estado = encontraSucessores(estado, encontraEstadoAtual(estado)) 
    return acharProximoSucessorNaoVisitado(estados_visitados, estado_com_sucessores_nao_visitados, sucessores_estado, 0)

### Metódo buscaEmProfundidade
O método executa a busca em profundidade a partir de uma matriz inicial e as execuções de trocas de lugar do 0, para encontrar a matriz objetivo (números ordenados de 1 a 8 e o 0 na posição (3, 3)).

##### Parâmetros: 
    matriz: array numpy que deve ser iterado - busca em seus sucessores - se não for igual ao desejado.
    
    matriz_objetivo: array numpy com array desejado.
    
    estados_visitados: estados que já foram visitados na busca.

##### Retorno: 
    bool: True se a matriz foi encontrada - atingiu a objetiva com as trocas de lugar. False se atingiu o máximo de iterações.
    
    matriz: matriz em seu estado final.
    


In [6]:
def buscaEmProfundidade(matriz, matriz_objetivo, estados_visitados): 
    global iteracao
    if (matriz == matriz_objetivo).all():
        return True, matriz
    elif iteracao == 2000: 
        return False, matriz
    else:
        sucessores = encontraSucessores(matriz, encontraEstadoAtual(matriz))
        for sucessor in sucessores: 
            if iteracao <= 2000:
                if not verificaSeFoiVisitado(sucessor, estados_visitados):
                    iteracao += 1
                    return buscaEmProfundidade(sucessor, matriz_objetivo, estados_visitados + [sucessor])
                else:
                    sucessor_valido = acharProximoSucessorNaoVisitado(estados_visitados, estados_visitados, sucessores, sucessores.index(sucessor))
                    iteracao += 1
                    return buscaEmProfundidade(sucessor_valido, matriz_objetivo, estados_visitados + [sucessor_valido])
            

### Metódo escreverArquivo
Escreve um relatório para comparação com o outro método em arquivo txt.

##### Parâmetros: 
    matriz_inicial: matriz de entrada.
    
    tempo: segundos para executar a busca com a matriz_inicial.
    
    iteracoes: iterações para executar a busca com a matriz_inicial.
    
    atingiu_objetivo: booleano que representa se a busca na matriz_inicial chegou ao estado final desejado.
 
##### Retorno: 
    nenhum. 

In [7]:
def escreverArquivo(matriz_inicial, tempo, iteracoes, atingiu_objetivo):
    arquivo = open('resultados_DFS.txt', 'a')
    
    arquivo.write(str(0))
    arquivo.write(';')
    
    arquivo.write(str(atingiu_objetivo))
    arquivo.write(';')
    
    arquivo.write(str(tempo))
    arquivo.write(';')
    
    arquivo.write(str(iteracoes))
    arquivo.write(';')
                  
    arquivo.write(str(matriz_inicial))
    arquivo.write(';*')

### Metódo executar
Executa a função da busca e escreve em um arquivo.

##### Parâmetros: 
    matriz_entrada: matriz de entrada para buscar seu estado igual à matriz_objetivo.
    
    matriz_objetivo: matriz para comparação, matriz_entrada deve ser igual à ela - idealmente.
 
##### Retorno: 
    nenhum. 

In [8]:
def executar(matriz_entrada, matriz_objetivo):
    inicio = time.time()
    retorno, matriz_final = buscaEmProfundidade(matriz_entrada, matriz_objetivo, [])
    fim = time.time()
    
    escreverArquivo(matriz_entrada, str(fim - inicio), str(iteracao), retorno)

In [9]:
def executarDFS(entrada, matriz_objetivo):
    global iteracao
    iteracao = 0
    executar(entrada, matriz_objetivo)